
## Modularity using the Karate network & GN algorithm

First, we will begin by installing the required packages in Python for our analysis:

In [ ]:
# Install networkx and install the required packages
!pip install -q networkx

import sys
import networkx as nx 
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import matplotlib as mpltlb

%matplotlib inline

We will then import the graph for Zachary's karate club, and retrieve the club labels for each node which indicate which side each member took:

In [ ]:
#Let's import the ZKC graph:
ZKC_graph = nx.karate_club_graph()

#Let's keep track of which nodes represent John A and Mr Hi
Mr_Hi = 0
John_A = 33

#Let's display the labels of which club each member ended up joining
club_labels = nx.get_node_attributes(ZKC_graph,'club')

#just show a couple of the labels
print({key:club_labels[key] for key in range(10,16)})

A = nx.convert_matrix.to_numpy_matrix(ZKC_graph)

We're going to apply an eigenvector-based method for identifying modules in the network, following a well-known paper by Girvan & Newman.

Newman, Mark EJ, and Michelle Girvan. "Finding and evaluating community structure in networks." *Physical Review E* 69.2 (2004): 026113.



In [ ]:
from networkx.algorithms.community.modularity_max import greedy_modularity_communities

# perform the community detection
c = list(greedy_modularity_communities(ZKC_graph))

# print the number of communities which were detected
print("Number of communities detected was", len(c))

In [ ]:
# let's print the members belonging to each cluster
community_0 = sorted(c[0])
community_1 = sorted(c[1])
community_2 = sorted(c[2])

print(community_0)
print(community_1)
print(community_2)

# join the two smaller communities
combined_community = community_1 + community_2


As we can see, $3$ communities were detected, and as we would hope the nodes corresponding to Mr. Hi ($0$) and John A ($33$) have been assigned to different communities. We've joined together the two smaller communities, because we know that the club only split into two parts.

We can visualise these results by plotting the communities as different colours. We will plot the nodes belonging to community 1 on the left, and the nodes belonging to community 2 on the right.

In [ ]:
# create the figure
fig, axs = plt.subplots()

# make a vector of locations for each of the nodes
location = np.zeros([34, 2])

# initially just assign them a random value
for n in range(34):
  location[n, :] = np.random.rand(1, 2)

# now add 3 to any of the entries in the combined community
for c in combined_community:
  location[c, 0] += 3;

# now plot all the members of each community - with one in green and the other in purple like the previous figure
axs.plot(location[community_0, 0], location[community_0, 1], 'go', markersize = 10)
axs.plot(location[combined_community, 0], location[combined_community, 1], 'mo', markersize = 10)

# now need to plot the connections between members
for i in range(34):
  for j in range(i, 34):

    # if a connection between i and j exists, plot the connection
    if A[i, j] == 1:
      axs.plot([location[i, 0], location[j, 0]], [location[i, 1], location[j, 1]], 'k', alpha = 0.5, linestyle='dashed', linewidth = 0.5)

# include some labels to make the figure look pretty
plt.xticks([0.5, 3.5], ["John A's club", "Mr. Hi's club"])
plt.yticks([])

# show the figure
plt.show()

By combining these two communities, we have split the network into two groups - the green, which we predict to join John A's club, and the purple which we predict ot join Mr. Hi's group. Comparing these assignments to the labels indicating the true groupings, we see that only one node has been incorrectly predicted, indicating a an accuracy rate of approximately $94$%.

